## Описание проекта

Сфера деятельности - страховая компания.

Необходимо защитить данные клиентов страховой компании. Нужно разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. 
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [5]:
df.info()
# Видно, что на первый взгляд данные адекватные 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [6]:
# Подготовим данные для дальнейших действий
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X X^T)^{-1} X^T y
$$

Задача: Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.) 

В нашем случае, признаки - это features (матрица, имеющая размерность 5000x4). Для решения этой задачи возьмем обратимую матрицу размером 4х4 (т.к в матрице features 4 столбца). 

**Ответ:** предположим, что качество линейной регрессии не изменится, то есть предсказания $a$ не меняются.

**Обоснование:** Известно, что матрица X - матрица признаков, поэтому домножим матрицу $X$ на обратимую матрицу $Z$, и подставим это произведение в $a$ и $w$.

$$a = XZw$$

$$w = ((XZ)^T XZ)^{-1}(XZ)^T y$$

Подставим $w$ в $a$:

$$a = XZ(((XZ)^T XZ)^{-1}(XZ)^T y)$$

Раскроем транспонирование по следующему свойству: $(XZ)^T = Z^T X^T$

$$a = XZ(((Z^T X^T XZ)^{-1}Z^T X^T y)$$

Используем свойство $(XZ)^{-1} = Z^{-1} X^{-1}$:

$$a = XZ(((X^T X Z)^{-1}(Z^T)^{-1} Z^T X^T y)$$

$$a = XZ Z^{-1}(((X^T X)^{-1} (Z^T)^{-1} Z^T X^T y)$$

Используем определение обратной матрицы:

$$a = XE(X^TX)^{-1}EX^Ty$$

Известно, что при умножении единичные матрицы сокращаются:

$$a = X(X^TX)^{-1}X^Ty$$

Как можно заметить, мы пришли к исходному виду $a$:

$$a = X(X^TX)^{-1}X^Ty = Xw$$

Таким образом, умножение признаков на обратимую матрицу не должно привести к изменению качества линейной регрессии.

## 3. Проверка алгоритма

In [7]:
# Посмотрим на метрику качества исходной матрицы
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print('Метрика качества R2 для модели с исходными признаками:',r2_score(target, predictions))

Метрика качества R2 для модели с исходными признаками: 0.42494550286668


In [8]:
# Сгенерируем случайную матрицу, имеющую размер 4х4
Z = np.random.normal(size = (4,4))
# Проверим её на обратимость
Z_invert = np.linalg.inv(Z)

In [9]:
# Умножим матрицу, содержащую признаки, на матрицу Z
# Для этого переведем features из датафрейма в матрицу 
X = features.values
# Умножим X на Z
new_features = np.dot(X, Z)

In [10]:
# Посмотрим изменилась ли на практике метрика качества при умножении матрицы признаков на обратимую матрицу
model = LinearRegression()
model.fit(new_features, target)
predictions = model.predict(new_features)
print('Метрика качества R2 для модели после умножения матрицы с признаками на обратимую матрицу Z:',r2_score(target, predictions))

Метрика качества R2 для модели после умножения матрицы с признаками на обратимую матрицу Z: 0.42494550286667965


## Вывод

Теоретически доказано, что произведение матрицы признаков на случайно сгенерированную обратимую матрицу не изменяет качество линейной регрессии. 

Практически получено, что действительно данная операция не приводит к изменению качества линейной регрессии (за метрику качества был принят коэффициент R2). 